# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Actividad Semana 5

### **Vectores Embebidos de OpenAI**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Elemento de lista
*   Elemento de lista



In [15]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd  
import numpy as np

import nltk
from nltk.corpus import stopwords
import re

from sklearn.model_selection import train_test_split

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

nltk.download('punkt')    # es un tokenizador que ayuda a dividr el texto en enunciados mediante un modelo no-supervisado.
nltk.download('stopwords')    # para tener acceso a "stopwords" en varios idiomas.
nltk.download('averaged_perceptron_tagger_eng')   # es un etiquetador de partes del discurso (POS) para inglés.
nltk.download('wordnet')   # es un recurso léxico para el inglés que agrupa palabras en conjuntos de sinónimos (synsets) y proporciona definiciones y ejemplos de uso.


[nltk_data] Downloading package punkt to /home/eahumada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eahumada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/eahumada/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/eahumada/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Incluye las celdas necesarias para tu acceso a la API de OpenAI.



# **Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [ ]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', delimiter='\t', names=['review','label'], header=None, encoding='utf-8')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')


df = pd.concat([dfa, dfi, dfy], ignore_index=True)
print(df.shape)

# *********** Aquí termina la sección de agregar código *************


(3000, 2)


In [3]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  3000 non-null   object 
 1   label   2000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 47.0+ KB


In [4]:
# Y veamos sus primeros registros:

df.head()

,review,label
0,So there is no way for me to plug it in here i...,0.0
1,"Good case, Excellent value.",1.0
2,Great for the jawbone.,1.0
3,Tied to charger for conversations lasting more...,0.0
4,The mic is great.,1.0


# **Pregunta - 2:**

Realiza el proceso de limpieza. Aplica el preprocesamiento que consideres adecuado.











In [10]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Definimos una lista de stopwords que no incluya las palabras negativas para evitar que se eliminen
# en el preprocesamiento, ya que pueden ser importantes para la clasificación.
negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
mystopwords = set(stopwords.words('english')) - set(negwords)


In [11]:
# Separamos nuestro dataset en dos variables, X e Y, donde X contiene las reseñas y Y contiene las etiquetas.
X = df.review     
Y = df.label      

# Verificamos que la longitud de X e Y sean correctas.
assert X.shape == (3000,)
assert Y.shape == (3000,)

In [12]:
# Definimos una función para limpiar el texto de las reseñas.
def clean_tok(doc):
  # Remplazamos los caracteres de puntuación por espacios.
  doc = re.sub(r'[^A-Za-z0-9]', ' ', doc)
  # Separamos el texto en tokens.
  as_list = doc.split()
  # eliminamos los caracteres no alfabéticos y convertimos a minúsculas.
  as_list = [re.sub(r'[^A-Za-z]+', '', s).lower() for s in as_list]
  # eliminamos las palabras de longitud menor a 2.
  as_list = [x for x in as_list if len(x) > 1]
  # eliminamos las stopwords.
  as_list = [x for x in as_list if x not in mystopwords]
  tokens = as_list

  return tokens

In [13]:
# Aplicamos 2 procesos extras de limpieza:
# 1. Eliminamos los caracteres repetidos (ej: "loooove" -> "loove").
# 2. Lematizamos las palabras (ej: "running" -> "run").
# Definimos una lista de palabras que no queremos lematizar.
# Esto es útil para evitar lematizar pronombres y otras palabras que no cambian su forma.
do_not_lemmatize = {
    'i', 'me', 'my', 'mine', 'you', 'your', 'yours',
    'he', 'him', 'his', 'she', 'her', 'hers',
    'we', 'us', 'our', 'ours', 'they', 'them', 'their', 'theirs',
    'it', 'its'
}

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Función para eliminar caracteres repetidos.
def reduce_repeated_chars(word):
    return re.sub(r'(.)\1{2,}', r'\1\1', word)

def clean_doc(doc):
    # eliminamos caracteres repetidos.
    reduced = [reduce_repeated_chars(word) for word in doc]
    # inicializamos el lematizador.
    lemmatizer = WordNetLemmatizer()
    # obtenemos las etiquetas de parte de la oración.
    pos_tags = pos_tag(reduced) 
    # lematizamos las palabras según su etiqueta.
    lemmatized = []
    for word, tag in pos_tags:
        lower_word = word.lower()
        if lower_word in do_not_lemmatize:
            lemmatized.append(lower_word)
        else:
            lemma = lemmatizer.lemmatize(word, get_wordnet_pos(tag))
            lemmatized.append(lemma)

    tokens = lemmatized

    return tokens

In [16]:
# Aplicamos el proceso de limpieza y tokenización:
Xcleantok = [clean_tok(x) for x in X]   

# Aplicamos el proceso de limpieza/normalización adicionales:
Xclean = [clean_doc(x) for x in Xcleantok]  


In [17]:
Xclean[0:5]         # Veamos los primero registros para ver cómo va quedando el resultado.

[['no', 'way', 'plug', 'us', 'unless', 'go', 'converter'],
 ['good', 'case', 'excellent', 'value'],
 ['great', 'jawbone'],
 ['tie', 'charger', 'conversation', 'last', 'minute', 'major', 'problem'],
 ['mic', 'great']]

In [ ]:
# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [18]:

# ************* Inicia la sección de agregar código:*****************************

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1) 
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)


# *********** Termina la sección de agregar código *************

# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


# **Pregunta - 4:**



Construye tu vocabulario a continuación


In [ ]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********


None


# *********** Aquí termina la sección de agregar código *************

c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [ ]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

# **Pregunta - 5:**


#### **Incluye aquí un resumen de las características y diferencias que tiene al menos los tres modelos de OpenAI indicados: "text-embedding-3-small", "text-embedding-3-large" y "text-embedding-ada-002".**

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


# **Pregunta - 6:**


#### **Diccionario clave-valor de palabras del diccionario y vectores embebidos.**

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 7:**



Generamos los vectores embebidos a partir de los conjuntos de entrenamiento, validación y prueba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

In [ ]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

# **Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 9:**



Reporte del mejor modelo con el conjunto de Prueba (Test).


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 10:**

In [ ]:
# Incluye todas las líneas de código y celdas que consideres adecuadas para este ejercicio.


None




# **Pregunta - 11:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

# **Fin de la Actividad de Vectores Embebidos - OpenAI**